# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

In [335]:
# modellazione del sistema
from dataclasses import dataclass
from typing import List, Tuple, Dict

@dataclass
class Binary:
    id: int
    services: List[str]
    working_on: int
    locked: bool

@dataclass
class Service:
    name: str
    binary: int
    impl_features: List[str]
    
@dataclass
class Feature:
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    completed: bool

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List]]

    def add_op(self, op_name: str, day: str, duration: int, values: List[int]):
        if day > self.avail:
            self.ops.append(("name", [day - self.avail]))
        self.avail += day + duration
        self.ops.append((op_name, values))

## Lettura input
In questa sezione, viene effettuato il parsing dei dati in ingresso creando:
* Lista dei binary: tutti i binary del sistema; per ogni binary si tiene traccia della list dei service installati e del numero di ingegneri che lavora al momento;
* Lista dei service; tutti i service del sistema. Per ogni service si tiene conto delle feature implementate al momento;
* Lista delle feature: tutte le feature del sistema. Ogni feature contiene una lista coppie (stringhe, booleani) in cui si tiene conto dei servizi da implementare e del loro stato (implementato si/no);

In [336]:
fname = "data/f.txt"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: metadati del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]

binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}
services: List[Service] = []
features: List[Feature] = []
engineers: List[Engineer] = []

for i in range(G):
    engineers.append(Engineer(i, 0, []))

for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [service], 0, False)
        binaries[bid] = b
    else:
        b.services.append(service)

    s = Service(service, bid, [])
    services.append(s)
    service_to_binaries[service] = bid

for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, services, difficulty, users = feature.strip().split(" ")
        services = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(feature, int(difficulty), int(users), services, False)
        features.append(f)

print(engineers, binaries, service_to_binaries, features, services, sep="\n")

[Engineer(id=0, avail=0, ops=[]), Engineer(id=1, avail=0, ops=[]), Engineer(id=2, avail=0, ops=[]), Engineer(id=3, avail=0, ops=[]), Engineer(id=4, avail=0, ops=[]), Engineer(id=5, avail=0, ops=[]), Engineer(id=6, avail=0, ops=[]), Engineer(id=7, avail=0, ops=[]), Engineer(id=8, avail=0, ops=[]), Engineer(id=9, avail=0, ops=[])]
[Binary(id=0, services=['sb', 'sbd', 'sbf', 'scc', 'scf', 'sea', 'seb', 'sfi', 'sjg'], working_on=0, locked=False), Binary(id=1, services=['si', 'sbj', 'sci', 'sdb', 'sdi', 'sfg', 'shh', 'sib', 'sid', 'sij'], working_on=0, locked=False), Binary(id=2, services=['sbb', 'scg', 'sdh', 'sec', 'sef', 'seg', 'sfc', 'sgh', 'shi', 'sia', 'sji'], working_on=0, locked=False), Binary(id=3, services=['sh', 'sj', 'sbg', 'sdc', 'sfa', 'sfe', 'sfh', 'sga', 'sgf', 'shg'], working_on=0, locked=False), Binary(id=4, services=['sa', 'sdd', 'sdf', 'sgj', 'sha', 'shj', 'sie', 'sjd'], working_on=0, locked=False), Binary(id=5, services=['se', 'sbc', 'scb', 'sch', 'sdg', 'sej', 'sff', '

## Attività
Di seguito sono illustrate le mosse che un ingegnere può effettuare.

Ciascuna mossa è caratterizzata da una durata. Per ogni mossa è definita una procedura per eseguirla sui dati di input. Ogni procedura ha la seguente firma:
* input:
    * ingegnere su cui schedulare l'attività;
    * giorno in cui effettuare l'attività;
    * feature, binary, service su cui effettuare l'attività;
* output:
    * durata dell'attività;

### Implementazione di una nuova feature
Su un *binary* $B_j$ può un ingegnere implementare tutti i servizi per una feature $F_i$. Questa mossa ha una durata $D_{Fi} + R_{Bj} + C_{Bj}$. Dove $D_{Fi}$ è data dalla complessità della **Feature**, $R_{Bj}$ è dato dal numero di servizi totali presenti nel *binary*, $C_{Bj}$ ingegneri che stanno già lavorando su $B_j$ nel giorno in cui viene *schedulata* l'attività. Inoltre, ogni ingegnere può implementare la stessa **Feature** su **binary** diversi. Infine, se più ingegneri sono *schedulati* per lavorare sullo stesso **binary** il primo della soluzione viene schedulato e conta per gli altri nel fattore $C_{Bj}$;


In [337]:
score: int = 0
daily_schedule: List[Tuple[int, int]] = [] # giorno, binary
def implement_feature(e: Engineer, day: int, f: Feature, b: int) -> int:
    global score
    duration  = f.difficulty + len(binaries[b].services) + binaries[b].working_on
    for service in f.dep_services:
        s_name = service[0]
        done = service[1]
        if service_to_binaries[s_name] == b and not done:
            service[1] = True
    
    for (service, done) in f.dep_services:
        if not done:
            break
    else:
        f.completed = True
        score += f.users * max(0, L-(day + duration))

    e.add_op("impl", day, duration, [f.name, b])
    daily_schedule.append((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

### Migrazione di un service
un ingegnere può spostare un servizio $S_i$ da un *binary* $B_j$ a $B_k$, mantenendo le funzionalità implementate. Per fare questa mossa, un ingegnere impiega $\max (R_{bj}, R_{bk})$ giorni dove $R_{bj}, R_{bk}$ sono i servizi in esecuzione rispettivamente su $B_j$ a $B_k$. Quest'azione però è bloccante: nessun ingegnere può lavorare su $B_j$ e $B_k$ quando è in corso una migrazione; 

In [338]:
def move_service(e: Engineer, day: int, s: str, dst: int) -> int:
    src = service_to_binaries[s]
    duration  = max(len(binaries[src]), len(binaries[dst]))
    service_to_binaries[s] = dst
    binaries[src].remove(s)
    binaries[dst].append(s)
    binaries[src].locked = True
    binaries[dst].locked = True
    e.add_op("move", day, duration, [s, dst])
    return duration

### Nuovo binary
Un ingegnere può creare un nuovo binary senza nessun servizio al suo interno. Il costo per creare è un nuovo binary è fisso e vale $N$.


In [339]:
def new_binary(e: Engineer, day: int)-> int:
    binaries.append(Binary(id))
    e.add_op("new", day, N, [])
    return N

### Idle
Un ingegnere può essere in idle per un numero di giorni $1\leq W \leq L$

In [340]:
def idle(e: Engineer, day: int, duration: int) -> int:
    e.add_op("wait", day, duration, [])
    return 1

# Approccio 1
In una fase preliminare, l'algoritmo ordina le feature per numero di utenti soddisfatti. In questo approccio, si assegna un punteggio maggiore ai servizi che risiedono sulla stessa macchina.

In [341]:
import random

for day in range(L):
    while len(daily_schedule) > 0 and daily_schedule[-1][0] <= day:
        bid: int = daily_schedule[-1][1] 
        binaries[bid].working_on -=1
        binaries[bid].locked = False
        daily_schedule.pop()

    for engineer in engineers:
        if engineer.avail <= day:
            b: Binary = random.choice(binaries)
            f: Feature = random.choice(features)
            print("implementing", f.name, "on", b.id)
            implement_feature(engineer, day, f, b.id)


# for engineer in engineers:
#     print(engineer.id)
#     for op in engineer.ops:
#         print(op)

print(score)
        

implementing siegacda on 9
implementing sfdefaac on 2
implementing sdiciaif on 3
implementing sccefged on 8
implementing sgijibdb on 8
implementing sigehcgb on 6
implementing sgfafgi on 7
implementing sbdajiec on 2
implementing sjejjhfa on 9
implementing sfjcfhji on 5
implementing sggcbgic on 1
implementing sfbgdbdf on 2
implementing scejgebh on 3
implementing sjagfhbi on 0
implementing siiaddih on 4
implementing sgehiihi on 9
implementing sebagbca on 4
implementing sbeeiccb on 3
implementing sjeibbba on 8
implementing sfbhhiif on 5
implementing sgbefach on 5
implementing shejehii on 9
implementing sicgiaaa on 9
implementing sjjbhcee on 4
implementing shbhcabb on 5
implementing shebeeaf on 2
implementing siecgfaa on 1
implementing sjdbejah on 2
implementing sdiigeed on 2
implementing sebabadi on 9
implementing sffhhgbe on 5
implementing shbhjjje on 0
implementing sedgfefd on 8
implementing sdhifdd on 0
implementing sdeaedci on 7
implementing shhfbjhj on 6
implementing scfdbacc on 1
imp